In [58]:
import pandas as pd 
import numpy as np
mzTol = 15
rtTol = .5


def same(feat1,feat2):
    mz1,rt1 = feat1
    mz2,rt2 = feat2
    if abs(rt1-rt2) < rtTol:
        if 1e6 * np.abs(mz1-mz2)/mz1 < mzTol:
            return True
    return False

In [62]:
#load in metadata and peak areas and associate

#read in peak data
datafilename = "../data/07_30_2020_batch_1_6_qTOF_polar_non_polar_peak_areas.csv"
data = pd.read_csv(datafilename)

#get area columns
areaCols = data.columns.values[5:]

datafilename = "../data/UNKOWNS_Polars_IdX_B1-6_WU350.csv"
dataUnk = pd.read_csv(datafilename)

#get area columns
areaColsUnk = dataUnk.columns.values[3:]
newNameMapping = {}
for samp in areaColsUnk:
    sampName = str(samp)
    sampName = sampName.replace("pns_","")

    newNameMapping[samp] = sampName

dataUnk = dataUnk.rename(newNameMapping,axis=1)
for x1 in range(len(dataUnk.columns.values)):
    for x2 in range(len(dataUnk.columns.values)):
        if x1 != x2:
            if dataUnk.columns.values[x1] == dataUnk.columns.values[x2]:
                print(dataUnk.columns.values[x1])
rows2Drop = []
for index,row in dataUnk.iterrows():
    pol = row["Polarity"]
    feat = (row["Mass"],row["RT"])
    if pol == "neg":
        filt = data[data["Compound Name"].str.contains("pHILICneg")]
        delta = -1.007
    elif pol == "pos":
        filt = data[data["Compound Name"].str.contains("pHILICpos")]
        delta = 1.007
    for index2,row2 in filt.iterrows():
        if same(feat,(row2["Mass"]+delta,row2["RT"])):
            rows2Drop.append(index)
            break
    dataUnk.at[index,"Compound Name"] = pol + "_" + dataUnk.at[index,"Compound Name"]
dataUnk = dataUnk.drop(rows2Drop)
print(len(rows2Drop))

200


In [70]:
combined = pd.concat((data[list(dataUnk.columns.values[1:4]) + list(areaCols)],dataUnk[dataUnk.columns.values[1:]]),axis=0)
combined.to_csv("../data/08_10_2020_batch_1_6_polar_non_polar_id_unk_peak_areas.csv",index=False)